<a href="https://colab.research.google.com/github/LennyRBriones/embeddings/blob/main/Embeddings_search_proyect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers
!pip install pinecone-client
!pip install gradio
!pip install typing-extensions --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=61f211e1ed42d75781ff9aa33ab1c3058e94f0ce0c868687ec7ebaa6bdf2108a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 4.6 MB

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from ast import literal_eval

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Models/25k-imdb-movie-dataset_6876f742-4304-4e96-b695-41e146a046c3.csv")

In [4]:
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/


Clean Data

In [5]:
def concatenate_list(list):
    list = literal_eval(list)
    return "".join(list)

In [6]:
def string_to_list(list):
    list = literal_eval(list)
    return list

In [7]:
df = df.fillna(" ")
df["Keywords"] = df["Plot Kyeword"].apply(concatenate_list)
df["Stars"] = df["Top 5 Casts"].apply(concatenate_list)
df["Generes"] = df["Generes"].apply(string_to_list)
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce").fillna(0).astype("float")

In [8]:
unique_generes = df["Generes"].explode().unique()
unique_generes

array(['Action', 'Drama', 'Adventure', 'Sci-Fi', 'Animation', 'Crime',
       'Comedy', 'Thriller', 'Fantasy', 'Horror', 'History', 'Mystery',
       'Biography', 'War', 'Western', 'Sport', 'Family', 'Romance',
       'Music', 'Musical', 'Film-Noir', 'Game-Show', 'Adult',
       'Reality-TV'], dtype=object)

In [9]:
df.drop(["Plot Kyeword", "Top 5 Casts"],axis=1, inplace=True)

In [10]:
df["text"] = df.apply(lambda x : str(x["Overview"])+ " " +x["Keywords"]+ " "+ x["Stars"], axis=1)

In [11]:
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Director,Writer,year,path,Keywords,Stars,text
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"[Action, Drama]",After more than thirty years of service as one...,Joseph Kosinski,Jim Cash,-2022,/title/tt1745960/,fighter jetsequelu.s. navyfighter aircraftpilo...,Jack Epps Jr.Peter CraigTom CruiseJennifer Con...,After more than thirty years of service as one...
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"[Action, Adventure, Sci-Fi]",Four years after the destruction of Isla Nubla...,Colin Trevorrow,Emily Carmichael,-2022,/title/tt8041270/,dinosaurjurassic parktyrannosaurus rexvelocira...,Colin TrevorrowDerek ConnollyChris PrattBryce ...,Four years after the destruction of Isla Nubla...
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"[Action, Drama]",As students at the United States Navy's elite ...,Tony Scott,Jim Cash,-1986,/title/tt0092099/,pilotmale camaraderieu.s. navygrumman f 14 tom...,Jack Epps Jr.Ehud YonayTom CruiseTim RobbinsKe...,As students at the United States Navy's elite ...


Moving the cleaned data set to a vectorial space

In [12]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [13]:
embeddings = model.encode(df['text'],batch_size=64,show_progress_bar=True)

Batches:   0%|          | 0/382 [00:00<?, ?it/s]

In [14]:
df["embeddings"] = embeddings.tolist()
df["ids"] = df.index
df["ids"] = df["ids"].astype("str")

In [15]:
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Director,Writer,year,path,Keywords,Stars,text,embeddings,ids
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"[Action, Drama]",After more than thirty years of service as one...,Joseph Kosinski,Jim Cash,-2022,/title/tt1745960/,fighter jetsequelu.s. navyfighter aircraftpilo...,Jack Epps Jr.Peter CraigTom CruiseJennifer Con...,After more than thirty years of service as one...,"[-0.04837854206562042, -0.003710306016728282, ...",0
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"[Action, Adventure, Sci-Fi]",Four years after the destruction of Isla Nubla...,Colin Trevorrow,Emily Carmichael,-2022,/title/tt8041270/,dinosaurjurassic parktyrannosaurus rexvelocira...,Colin TrevorrowDerek ConnollyChris PrattBryce ...,Four years after the destruction of Isla Nubla...,"[-0.030405916273593903, -0.018286526203155518,...",1
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"[Action, Drama]",As students at the United States Navy's elite ...,Tony Scott,Jim Cash,-1986,/title/tt0092099/,pilotmale camaraderieu.s. navygrumman f 14 tom...,Jack Epps Jr.Ehud YonayTom CruiseTim RobbinsKe...,As students at the United States Navy's elite ...,"[0.026411941275000572, 0.05586894601583481, -0...",2


In [16]:
import pinecone
from getpass import getpass

In [17]:
pinecone_api = getpass("Enter the value...")

Enter the value...··········


In [18]:
pinecone.init(api_key= pinecone_api, environment="gcp-starter")

In [19]:
dimension_embeddings= len(df["embeddings"][0])
index_name = "movies-embeddings"
all_index = pinecone.list_indexes()
if index_name in all_index:
  index = pinecone.Index(index_name)
else:
  pinecone.create_index(index_name, dimension= dimension_embeddings, metric="cosine")
  index = pinecone.Index(index_name)


In [20]:
index

In [21]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df[i:i_end]
    # generate embeddings for batch
    ids = batch['ids']
    emb = batch['embeddings']
    metadata = batch.drop(['ids','embeddings','text','path'],axis=1).to_dict('records')

    # add all to upsert list
    to_upsert = list(zip(ids, emb,metadata))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/382 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.24402,
 'namespaces': {'': {'vector_count': 24402}},
 'total_vector_count': 24402}

In [26]:
query = 'a history of time travel and science'
query_vector = model.encode(query).tolist()

responses = index.query(
  vector=query_vector,
  top_k=3,
  include_metadata=True,
  filter ={
  "Generes": { "$in": ['Action'] }
}
)

In [27]:
responses

{'matches': [{'id': '3387',
              'metadata': {'Director': 'Vikram K. Kumar',
                           'Generes': ['Action', 'Comedy', 'Drama'],
                           'Keywords': 'brothertime travelwatchtime traveler',
                           'Overview': 'A scientist invents a time machine, '
                                       'which leads to a bitter battle between '
                                       'his evil twin brother and his son.',
                           'Rating': 7.9,
                           'Run Time': datetime.datetime(2023, 11, 4, 2, 44),
                           'Stars': 'SuriyaSamantha Ruth PrabhuNithya '
                                    'MenenVikram K. KumarVikram K. Kumar',
                           'User Rating': '22K',
                           'Writer': 'Vikram K. Kumar',
                           'movie title': '24',
                           'year': '(I) (2016)'},
              'score': 0.626256347,
              'values': 

In [28]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.24402,
 'namespaces': {'': {'vector_count': 24402}},
 'total_vector_count': 24402}

In [29]:
def search(query, genre, rating, top_k):
  query_vector = model.encode(query).tolist()

  if rating:
      filter_rating = rating
  else:
      filter_rating = 0
  if genre:
      conditions = {
          "Generes" : {
              "$in" : [genre]

          },
          "Rating" : {
              "$gte": filter_rating
          }
      }
  else:
      conditions = {
          "Rating": {
              "$gte": filter_rating
          }
      }

  responses = index.query(
      vector = query_vector,
      top_k = top_k,
      include_metadata=True,
      filkter = conditions
  )
  response_data = []
  for response in responses["matches"]:
    response_data.append(
        {
            "Title" : response["metadata"]["movie title"],
            "Overview" : response["metadata"]["Overview"],
            "Director" : response["metadata"]["Director"],
            "Genre" : response["metadata"]["Generes"],
            "year" : response["metadata"]["year"],
            "Rating" : response["metadata"]["Rating"],

        }
    )

  df = pd.DataFrame(response_data)
  return df


In [30]:
search("A time travel adventure",None,None,5)

,Title,Overview,Director,Genre,year,Rating
0,The Exotic Time Machine,A time machine transports two curious pleasure...,Felicia Sinclair,[Sci-Fi],-1998,3.6
1,Timeline,A group of archaeologists become trapped in th...,Richard Donner,"[Action, Adventure, Sci-Fi]",-2003,5.6
2,Norman,A time traveler and his A.I. companion. Norman...,Joel Guelzo,"[Adventure, Sci-Fi, Thriller]",(IV) (2019),3.6
3,A Sound of Thunder,"A single mistake in the past, by a time travel...",Peter Hyams,"[Action, Adventure, Horror]",-2005,4.2
4,Aditya 369,A group of children use a time machine to inve...,Singeetam Srinivasa Rao,"[Adventure, Comedy, Sci-Fi]",-1991,8.5


In [31]:
import gradio as gr

# Define possible genres
genres = unique_generes.tolist()
iface = gr.Interface(
    fn=search,
    inputs=[
        gr.Textbox(lines=5, placeholder="Type here any to search...", label="Search"),
        gr.Dropdown(choices=genres, label="Movie Genre"),
        gr.Slider(minimum=1, maximum=10, value=5, label="Minimum Score"),
        gr.Number(minimum=1, maximum=10, value=3, label="Number of movies to search")

    ],
    outputs=gr.Dataframe(type="pandas", label="Results"),
    title="Movie Searcher",
    description="type the search, select the genre and define the minimum rank to search movies.",
)

# Launch the interface
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c018c644518c05502a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
